In [ ]:
"""
Цель данного ноутбука, улучшить модель из предыдущей части.Конкретнее я планирую работать с 1 моделью - рекомендация через поиск 
похожих пользователей. Я запланировал комплексное улучшение модели, во первых усложнение архитектуры. Мы будем аналогичным образом просматривать
библиотеку пользователя и искать похожие игры которые его могут заинтересовать. В итоге наше предложение пользователю будет состоять как и из игр
игр имеющихся у похожих на него потребителей, так и из игр похожие на те в которые он играл. Таким образом мы введём ансамблево гибридный метод
создания предсказаний.

Также я переосмыслил метрики оценки проекта, для того что бы оценить качество работы модели. Мы воспользуемся Аугментацией и построим новых
пользователей, для создание более удобных условий тестировани. Например мы можем создать нового пользователя удалив часть игр из библиотеки и посмотреть
будет ли в рекомендациях модели те игры которые были у оригинального пользователя, а у аугментированной версии отсутствовали. В классическом смысле
аугментация нам не пригодиться, так как мы не работаем с большими массивами текстов/фото.

По аналогичным причинам я не вижу корректных способов работать над созданием синтетического датасета, пусть даже частичного разбаляя реальный датасет
вымышленными банными, так как у нас нет возможности симулировать достоверных пользователей с теми же предпочтениями, что и у оригинала.

Также в качестве улучшения модели я планирую использовать другой подход для выбора фичей, фичи которые показали себя как мало влияющие на результат
будут отброщены, в то время как серьёзные фичи переработанны, каким образом подробнее будет описанно далее, во время сообственно работы с данными.
Ещё я планирую частично переработать подход к подготовке данных, для более успешного создания модели.
"""

In [2]:
"""
Часть 1
Имплементируем Feedback 4 части. Обучим обычную логистическую регрессию на табличке. Описания игр не будем считать. Сравним по метрикам из 4 части.
Это поможет нам более комплексно оценить имеющуюся у нас модель.
"""

1


In [62]:
#Начнём с импортирования необходимых нам библиотек
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer


In [63]:
#импортируем данные и проводим над ними теже манипуляции, что и в 4 части.
df_games = pd.read_csv("output_file_games.csv")

df_games['release_date'] = pd.to_datetime(df_games['release_date'], errors='coerce')
df_games = df_games.dropna(subset=['release_date'])

df_games['price'] = df_games['price'].replace({'€': '', '': ''}, regex=True) 
df_games['price'] = df_games['price'].replace({',': '.', '': ''}, regex=True) 
df_games['price'] = df_games['price'].replace({'Free': '0', '': ''}, regex=True) 
df_games['price'] = pd.to_numeric(df_games['price'], errors='coerce')
df_games = df_games.dropna(subset=['price'])
df_games['price'] = df_games['price'].astype(float)

df_games['release_date'] = df_games['release_date'].dt.year

df_games.head()

,name,steam_appid,description,release_date,developers,publishers,genres,price
0,Aftermath Y2K,655360,Inspired by such games as Castlevania and Ninj...,2017,"['Matthew Willis', 'Jesse Crespo']",['Matthew Willis'],"['Action', 'Indie']",3.99
1,Super Man Or Monster,393220,Super Man Or Monster is an explosive monsterfi...,2017,['Xform'],['Xform'],"['Action', 'Adventure', 'Indie']",12.99
2,Terje Haakonsen's Powder VR,1048580,"Master the mountain from 14,000 ft in this ext...",2020,['RainSoft'],['RainSoft'],"['Casual', 'Indie', 'Racing', 'Simulation', 'S...",16.79
3,Vanguard Princess,262150,Vanguard Princess is a classic 2D fighting gam...,2014,['Tomoaki Sugeno'],['eigoMANGA'],"['Action', 'Casual', 'Indie']",4.99
5,Train Bandit,655370,THE LAW!! Looks to be a showdown atop a train....,2017,['Rusty Moyher'],['Wild Rooster'],"['Action', 'Indie']",0.99


In [64]:
df_games = df_games.drop("description", axis = 1)

In [65]:
df_games.head()

,name,steam_appid,release_date,developers,publishers,genres,price
0,Aftermath Y2K,655360,2017,"['Matthew Willis', 'Jesse Crespo']",['Matthew Willis'],"['Action', 'Indie']",3.99
1,Super Man Or Monster,393220,2017,['Xform'],['Xform'],"['Action', 'Adventure', 'Indie']",12.99
2,Terje Haakonsen's Powder VR,1048580,2020,['RainSoft'],['RainSoft'],"['Casual', 'Indie', 'Racing', 'Simulation', 'S...",16.79
3,Vanguard Princess,262150,2014,['Tomoaki Sugeno'],['eigoMANGA'],"['Action', 'Casual', 'Indie']",4.99
5,Train Bandit,655370,2017,['Rusty Moyher'],['Wild Rooster'],"['Action', 'Indie']",0.99


In [66]:
#Get dummies OHE для тэгов/жанров
df_games['genres'] = df_games['genres'].apply(lambda x: x[1:-1].split(',') if x[0] == '[' and x[-1] == ']' else x.split(','))

df_genres = df_games.explode('genres')
df_genres['genres'] = df_genres['genres'].str.strip()
df_encoded = pd.get_dummies(df_genres['genres'])

df_final = df_encoded.groupby(df_genres.index).max()
df_result = df_games.join(df_final)

df_result.head(5)

,name,steam_appid,release_date,developers,publishers,genres,price,,'Accounting','Action',...,'RPG','Racing','Simulation','Software Training','Sports','Strategy','Utilities','Video Production','Violent','Web Publishing'
0,Aftermath Y2K,655360,2017,"['Matthew Willis', 'Jesse Crespo']",['Matthew Willis'],"['Action', 'Indie']",3.99,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,Super Man Or Monster,393220,2017,['Xform'],['Xform'],"['Action', 'Adventure', 'Indie']",12.99,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,Terje Haakonsen's Powder VR,1048580,2020,['RainSoft'],['RainSoft'],"['Casual', 'Indie', 'Racing', 'Simulation',...",16.79,False,False,False,...,False,True,True,False,True,False,False,False,False,False
3,Vanguard Princess,262150,2014,['Tomoaki Sugeno'],['eigoMANGA'],"['Action', 'Casual', 'Indie']",4.99,False,False,True,...,False,False,False,False,False,False,False,False,False,False
5,Train Bandit,655370,2017,['Rusty Moyher'],['Wild Rooster'],"['Action', 'Indie']",0.99,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [67]:
df_result = df_result.drop('genres', axis = 1)

In [68]:
#Информацию об издателях будем использовать агрегатною частоту появления тех или иных издателей, по сути тот же TF-IDF, 
#только суммированный, да частью информации придёться пожертвовать, но такога издержка использования логистической регрессии.
#сообственна она нужна только, как было сказано раннее для сравнения эффективности метрик.

In [69]:
tfidf = TfidfVectorizer()

tfidf_matrix = tfidf.fit_transform(df_result['publishers'])
tfidf_sums = tfidf_matrix.sum(axis=1)
df_tfidf_sums = pd.DataFrame(tfidf_sums, columns=["TF-IDF"])


df_result['publishers'] = df_tfidf_sums

In [70]:
tfidf_matrix = tfidf.fit_transform(df_result['developers'])
tfidf_sums = tfidf_matrix.sum(axis=1)
df_tfidf_sums = pd.DataFrame(tfidf_sums, columns=["TF-IDF"])


df_result['developers'] = df_tfidf_sums

In [71]:
df_result.head()

,name,steam_appid,release_date,developers,publishers,price,,'Accounting','Action','Adventure',...,'RPG','Racing','Simulation','Software Training','Sports','Strategy','Utilities','Video Production','Violent','Web Publishing'
0,Aftermath Y2K,655360,2017,1.997121,1.411091,3.99,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,Super Man Or Monster,393220,2017,1.000000,1.000000,12.99,False,False,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Terje Haakonsen's Powder VR,1048580,2020,1.000000,1.000000,16.79,False,False,False,False,...,False,True,True,False,True,False,False,False,False,False
3,Vanguard Princess,262150,2014,1.414214,1.000000,4.99,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
5,Train Bandit,655370,2017,1.414214,1.397992,0.99,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [72]:
pd.set_option('future.no_silent_downcasting', True)

In [73]:
#придадим ОНЕ переменным значения 0 и 1, вместо булеанов
df_result = df_result.replace({True: 1, False: 0})
df_result.head()

,name,steam_appid,release_date,developers,publishers,price,,'Accounting','Action','Adventure',...,'RPG','Racing','Simulation','Software Training','Sports','Strategy','Utilities','Video Production','Violent','Web Publishing'
0,Aftermath Y2K,655360,2017,1.997121,1.411091,3.99,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Super Man Or Monster,393220,2017,1.000000,1.000000,12.99,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,Terje Haakonsen's Powder VR,1048580,2020,1.000000,1.000000,16.79,0,0,0,0,...,0,1,1,0,1,0,0,0,0,0
3,Vanguard Princess,262150,2014,1.414214,1.000000,4.99,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,Train Bandit,655370,2017,1.414214,1.397992,0.99,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
#последние приготовления
df_result = df_result.drop('name', axis = 1)
df_result['release_date'] =  df_result['release_date'] - 2000
df_result.head()

,steam_appid,release_date,developers,publishers,price,,'Accounting','Action','Adventure','Animation & Modeling',...,'RPG','Racing','Simulation','Software Training','Sports','Strategy','Utilities','Video Production','Violent','Web Publishing'
0,655360,17,1.997121,1.411091,3.99,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,393220,17,1.000000,1.000000,12.99,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1048580,20,1.000000,1.000000,16.79,0,0,0,0,0,...,0,1,1,0,1,0,0,0,0,0
3,262150,14,1.414214,1.000000,4.99,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,655370,17,1.414214,1.397992,0.99,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
df_result['release_date'] =  df_result['release_date']/10.0
df_result.head()

,steam_appid,release_date,developers,publishers,price,,'Accounting','Action','Adventure','Animation & Modeling',...,'RPG','Racing','Simulation','Software Training','Sports','Strategy','Utilities','Video Production','Violent','Web Publishing'
0,655360,1.7,1.997121,1.411091,3.99,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,393220,1.7,1.000000,1.000000,12.99,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1048580,2.0,1.000000,1.000000,16.79,0,0,0,0,0,...,0,1,1,0,1,0,0,0,0,0
3,262150,1.4,1.414214,1.000000,4.99,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,655370,1.7,1.414214,1.397992,0.99,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
#Далее мы уже можем проводить merge по steam_appid в df_result и app_id в df_new с колонкой.
#Так же часть числовой информации о самих пользователях перенесём от туда же. Работать будет аналогично с плейтаймом.
#Но в рамках улучшения метрики, и борьбой с "серединной" проблемой она будет бинарной

In [91]:
df_new = pd.read_csv('part4_transformed_data.csv')
#df_new.head()

In [93]:
temp1 = [
    'UserID', 
    'app_id', 
    'playtime', 
    'games_owned_num', 
    'level', 
    'badge_total', 
    'current_xp', 
    'needed_xp', 
    'friends'
]

df_new = df_new[temp1]
#df_new.head()

In [94]:
df_new['playtime'] = pd.DataFrame([0 if val < 0 else 1 for val in df_new['playtime']])
df_new.head()

,UserID,app_id,playtime,games_owned_num,level,badge_total,current_xp,needed_xp,friends
0,76561198119601974,655360,0,1760,17.0,10.0,2467.0,0.0,236
1,76561198119601974,393220,0,1760,17.0,10.0,2467.0,0.0,236
2,76561198119601974,1048580,0,1760,17.0,10.0,2467.0,0.0,236
3,76561198119601974,262150,1,1760,17.0,10.0,2467.0,0.0,236
4,76561198119601974,655370,0,1760,17.0,10.0,2467.0,0.0,236


In [95]:
merged_df = df_new.merge(df_result, left_on='app_id', right_on='steam_appid', how='left')
merged_df.head()

,UserID,app_id,playtime,games_owned_num,level,badge_total,current_xp,needed_xp,friends,steam_appid,...,'RPG','Racing','Simulation','Software Training','Sports','Strategy','Utilities','Video Production','Violent','Web Publishing'
0,76561198119601974,655360,0,1760,17.0,10.0,2467.0,0.0,236,655360,...,0,0,0,0,0,0,0,0,0,0
1,76561198119601974,393220,0,1760,17.0,10.0,2467.0,0.0,236,393220,...,0,0,0,0,0,0,0,0,0,0
2,76561198119601974,1048580,0,1760,17.0,10.0,2467.0,0.0,236,1048580,...,0,1,1,0,1,0,0,0,0,0
3,76561198119601974,262150,1,1760,17.0,10.0,2467.0,0.0,236,262150,...,0,0,0,0,0,0,0,0,0,0
4,76561198119601974,655370,0,1760,17.0,10.0,2467.0,0.0,236,655370,...,0,0,0,0,0,0,0,0,0,0


In [97]:
#можем дропнуть app_id и UserID так как уже все необходимые данные уже перенесенны
merged_df = merged_df.drop("UserID", axis = 1)
merged_df = merged_df.drop("app_id", axis = 1)
merged_df.head()

,playtime,games_owned_num,level,badge_total,current_xp,needed_xp,friends,steam_appid,release_date,developers,...,'RPG','Racing','Simulation','Software Training','Sports','Strategy','Utilities','Video Production','Violent','Web Publishing'
0,0,1760,17.0,10.0,2467.0,0.0,236,655360,1.7,1.997121,...,0,0,0,0,0,0,0,0,0,0
1,0,1760,17.0,10.0,2467.0,0.0,236,393220,1.7,1.000000,...,0,0,0,0,0,0,0,0,0,0
2,0,1760,17.0,10.0,2467.0,0.0,236,1048580,2.0,1.000000,...,0,1,1,0,1,0,0,0,0,0
3,1,1760,17.0,10.0,2467.0,0.0,236,262150,1.4,1.414214,...,0,0,0,0,0,0,0,0,0,0
4,0,1760,17.0,10.0,2467.0,0.0,236,655370,1.7,1.414214,...,0,0,0,0,0,0,0,0,0,0


In [98]:
merged_df['games_owned_num'] = merged_df['games_owned_num']/50.0
merged_df['current_xp'] = merged_df['current_xp']/1000.0
merged_df['badge_total'] = merged_df['badge_total']/12.0
merged_df['level'] = merged_df['level']/25.0
merged_df['friends'] = merged_df['friends']/50.0
merged_df = merged_df.drop("steam_appid", axis = 1)
merged_df.head()

,playtime,games_owned_num,level,badge_total,current_xp,needed_xp,friends,release_date,developers,publishers,...,'RPG','Racing','Simulation','Software Training','Sports','Strategy','Utilities','Video Production','Violent','Web Publishing'
0,0,35.2,0.68,0.833333,2.467,0.0,236,1.7,1.997121,1.411091,...,0,0,0,0,0,0,0,0,0,0
1,0,35.2,0.68,0.833333,2.467,0.0,236,1.7,1.000000,1.000000,...,0,0,0,0,0,0,0,0,0,0
2,0,35.2,0.68,0.833333,2.467,0.0,236,2.0,1.000000,1.000000,...,0,1,1,0,1,0,0,0,0,0
3,1,35.2,0.68,0.833333,2.467,0.0,236,1.4,1.414214,1.000000,...,0,0,0,0,0,0,0,0,0,0
4,0,35.2,0.68,0.833333,2.467,0.0,236,1.7,1.414214,1.397992,...,0,0,0,0,0,0,0,0,0,0


In [197]:
#Создаём сбалансированный train test split
filtered_df = merged_df[merged_df['playtime'] == 0.0]
filtered_df2 = merged_df[merged_df['playtime'] == 1.0]
filtered_df = filtered_df.sample(n=len(filtered_df2), random_state=42)

print(filtered_df2.head())

filtered_df2 = pd.concat([filtered_df, filtered_df2], ignore_index=True)

#*2 так как в реальности большинство данных это 0. Мы не должны создать слишком оптимистичную модель. То есть мы сейчас в гибридном состояние, мы невелируем сильный разрыв между 2 категориями будущего таргета, но оставляем его в контролируемой форме. Надо просто удебиться в том что эффективноть выше нежеле чем в праиле 0. (Всегда 0 - > Будет выдавать до 2/3 эффективноти)

filtered_df2['playtime'].value_counts()

     playtime  games_owned_num  level  badge_total  current_xp  needed_xp  \
3         1.0             35.2   0.68     0.833333       2.467        0.0   
170       1.0             35.2   0.68     0.833333       2.467        0.0   
194       1.0             35.2   0.68     0.833333       2.467        0.0   
290       1.0             35.2   0.68     0.833333       2.467        0.0   
317       1.0             35.2   0.68     0.833333       2.467        0.0   

     friends  release_date  developers  publishers  ...  'RPG'  'Racing'  \
3      236.0           1.4    1.414214    1.000000  ...    0.0       0.0   
170    236.0           1.4    1.411091    1.411091  ...    0.0       1.0   
194    236.0           1.1    1.414214    1.900939  ...    0.0       0.0   
290    236.0           1.5    1.414214    1.414214  ...    0.0       0.0   
317    236.0           1.3    1.363157    1.363157  ...    1.0       0.0   

     'Simulation'  'Software Training'  'Sports'  'Strategy'  'Utilities'  \
3  

playtime
0.0    45047
1.0    45047
Name: count, dtype: int64

In [198]:
#filtered_df.fillna(0)
filtered_df.head(1)

,playtime,games_owned_num,level,badge_total,current_xp,needed_xp,friends,release_date,developers,publishers,...,'RPG','Racing','Simulation','Software Training','Sports','Strategy','Utilities','Video Production','Violent','Web Publishing'
2663336,0.0,43.22,1.28,3.666667,6.846,0.0,56.0,1.8,1.0,1.414214,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [199]:
filtered_df.astype(float)
for column in filtered_df:
    filtered_df[column] = filtered_df[column].astype(float)
print("debug")
filtered_df.head(1)
filtered_df.dropna(inplace=True)

debug


In [163]:
#filtered_df = filtered_df.drop("", axis = 1)
for column in merged_df:
    merged_df[column] = merged_df[column].astype(float)
merged_df.dropna(inplace=True)

In [164]:
from sklearn.model_selection import train_test_split

X = merged_df.drop(columns=['playtime'])
y = merged_df['playtime']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [165]:
(X_train.dtypes)
X_train.isna().sum()

games_owned_num            0
level                      0
badge_total                0
current_xp                 0
needed_xp                  0
friends                    0
release_date               0
developers                 0
publishers                 0
price                      0
                           0
'Accounting'               0
'Action'                   0
'Adventure'                0
'Animation & Modeling'     0
'Audio Production'         0
'Casual'                   0
'Design & Illustration'    0
'Early Access'             0
'Education'                0
'Free To Play'             0
'Game Development'         0
'Gore'                     0
'Indie'                    0
'Massively Multiplayer'    0
'Photo Editing'            0
'RPG'                      0
'Racing'                   0
'Simulation'               0
'Software Training'        0
'Sports'                   0
'Strategy'                 0
'Utilities'                0
'Video Production'         0
'Violent'     

In [166]:
#Собственно теперь сама модель

from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [167]:
y_pred = model.predict(X_test)

In [168]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Absolute Error: 0.012887307305896854
Mean Squared Error: 0.004760568376548727
R-squared: 0.13250840191168078


In [206]:
#Результаты на несбалансированном датасете выглядят хорошо, но на деле модель просто нашла правило - 0. И делает наиболее успешные предположение независимо от ввода.
#Повторим на сбалансированном.

1741229    0.0
302932     0.0
6950396    0.0
7234397    0.0
430715     0.0
7487597    0.0
4153057    0.0
7527573    0.0
2580530    0.0
5911426    0.0
Name: playtime, dtype: float64

In [207]:
from sklearn.model_selection import train_test_split

X = filtered_df2.drop(columns=['playtime'])
y = filtered_df2['playtime']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [208]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [209]:
y_pred = model.predict(X_test)

In [210]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Absolute Error: 0.305802976068755
Mean Squared Error: 0.13640817010706552
R-squared: 0.4543375915543083


In [ ]:
#Как видим ошибка возрасла и стала весьма серьёзной - до 0.3. То есть из 19 предсказаний 3 ошибочны.

In [ ]:
#Следовательно мы можем придти к заключению, что построение модели классической регрессие недоходит до уровня которго мы смогли добиться комбинированным подходом в предыдущей части
#И в заключение упор дложен делаться на хорошо себя показавшую в предыдущей части проекта полносвязную нейронную сеть глубинного обучения.
#С мультимодальной обработкой признаков.

In [ ]:
"""
Однако несмотря ни на что в этой части мы смогли существенно расширить пайплайн предобработки засчёт объеденения таблиц игр и пользователей.
Теперь размерность датасета user_id * app_id что делает размерность более чем достаточной для наших целей, хоть и повышает sparcity

Также мы смогли подробнее оценить метрику (оценки предсказаний). Сравнивая полученный результат не только с банальным правилом 0, но также и с
результатами классической линейной регрессии. Это дало нам много полезной информации о качестве модели и определило путь дальнейшей разработки.
"""